In [152]:
import os

import numpy as np
import pandas as pd
import seaborn as sns


import matplotlib.pyplot as plt
import utils
from specstan.plot import scatter_combined, default_settings
from utils import load_stan_model
from sklearn.manifold import Isomap, TSNE
import extinction

In [ ]:
STAN_MODEL = load_stan_model("./stan_models/read_between_the_lines.stan")

In [ ]:
STAN_MODEL[1].show()

In [ ]:
"""
StanModel object 'anon_model_b8c59412995a2cb1104bb18453c94b02' coded as follows:
data {
    int<lower=0> num_targets;
    int<lower=0> num_wave;
    vector[num_wave] maximum_flux[num_targets];
    vector[num_wave] maximum_fluxerr[num_targets];
    vector[num_wave] color_law;
}
transformed data{
    // Sum-to-zero transformations
    matrix[num_targets, num_targets] sum_zero_mat =
        diag_matrix(rep_vector(1, num_targets));
    matrix[num_targets, num_targets-1] sum_zero_qr;
    for (i in 1:num_targets-1) sum_zero_mat[num_targets,i] = -1;
    sum_zero_mat[num_targets, num_targets] = 0;
    sum_zero_qr = qr_Q(sum_zero_mat)[ , 1:(num_targets-1)];
}
parameters {
    vector[num_wave] mean_flux;
    vector<lower=0>[num_wave] fractional_dispersion;

    vector[num_targets-1] colors_raw;
    vector[num_targets-1] magnitudes_raw;
}
transformed parameters {
    vector[num_wave] model_diffs[num_targets];
    vector[num_wave] model_scales[num_targets];
    vector[num_wave] model_flux[num_targets];
    vector[num_wave] model_fluxerr[num_targets];

    vector[num_targets] colors = sum_zero_qr * colors_raw;
    vector[num_targets] magnitudes = sum_zero_qr * magnitudes_raw;

    for (t in 1:num_targets) {
        model_diffs[t] = magnitudes[t] + color_law * colors[t];
        model_scales[t] = exp(-0.4 * log(10) * model_diffs[t]);
        model_flux[t] = mean_flux .* model_scales[t];
        model_fluxerr[t] = sqrt(
            square(maximum_fluxerr[t])
            + square(fractional_dispersion .* model_flux[t])
        );
    }
}
model {
    colors ~ normal(0, 1.0);
    for (t in 1:num_targets) {
        maximum_flux[t] ~ normal(model_flux[t], model_fluxerr[t]);
    }
}

"""

In [ ]:
def load_rbtl_dispersion(file_path):
    names = [
        "sn", "salt2_x1", "salt2_x1_unc", "salt2_c", "salt2_c_unc",
        "delta_av", "delta_av_unc", "delta_m", "delta_m_unc", "xi1", "xi2",
        "xi3"
    ]
    dtypes = {key: np.float64 for key in names}
    dtypes["sn"] = str
    rbtl_data = pd.read_csv(file_path, sep="\s+", names=names, header=0,
                            dtype=dtypes)
    return rbtl_data


def load_global_params(file_path):
    names = ["wavelength", "c1", "c2", "eps_neg_5", "eps_neg_2_5", "eps_2_5",
             "eps_5", "eta"]
    dtypes = {key: np.float64 for key in names}
    dtypes["wavelength"] = np.int64
    global_params = pd.read_csv(file_path, sep="\s+", names=names, header=0,
                                dtype=dtypes)
    return global_params


RBTL_DATA = load_rbtl_dispersion("../SNfactory/rbtl-dispertion.txt")
GLOBAL_PARAMS = load_global_params("../SNfactory/global-diff-rbtl-params.txt")

In [ ]:
def load_flux(dir_path, sn_order):
    spectra = {}
    for spectrum_f_path in os.scandir(dir_path):
        sn = spectrum_f_path.name.split(".")[0]
        spectra[sn] = np.loadtxt(spectrum_f_path, skiprows=1)[None, ...]
    print(len(spectra))
    spectra = [spectra[sn] for sn in sn_order]
    print(len(spectra))
    spectra = np.concatenate(spectra, axis=0)
    return spectra[..., 0], spectra[..., 1], spectra[..., 2]


WAVELENGTHS, SPECTRA, SPECTRA_ERR = load_flux("../SNfactory/spectra",
                                              RBTL_DATA["sn"].values)

In [149]:
def recover_pre_rbtl_spectra(wavelengths, spectra, mag_residual, rbtl_extinction):
    rbtl_color_law = extinction.fitzpatrick99(
        wavelengths, 1.0, default_settings['rbtl_fiducial_rv'] # 2.8
    )
    return spectra / 10 ** (0.4 * (mag_residual[:, None] + rbtl_extinction[:, None] * rbtl_color_law))

RECOVERED_SPECTRA = recover_pre_rbtl_spectra(
    WAVELENGTHS[0], SPECTRA, RBTL_DATA["delta_m"].values, RBTL_DATA["delta_av"].values
)

In [139]:
def get_noise_mask(global_params, spectra, spectra_err, max_frac=0.1):
    intrinsic_dispersion = global_params["eta"].values
    intrinsic_power = np.sum(intrinsic_dispersion ** 2)
    maximum_uncertainty = utils.frac_to_mag(
        spectra_err / spectra
    )
    maximum_power = np.sum(maximum_uncertainty ** 2, axis=1)
    maximum_uncertainty_fraction = maximum_power / intrinsic_power
    return maximum_uncertainty_fraction < max_frac

def get_extinction_mask(extinction, max_extinction=0.5):
    return np.abs(extinction) < max_extinction
    

NOISE_MASK = get_noise_mask(GLOBAL_PARAMS, SPECTRA, SPECTRA_ERR)
EXTINCTION_MASK = get_extinction_mask(RBTL_DATA["delta_av"].values)
COMBINED_MASK = np.logical_and(NOISE_MASK, EXTINCTION_MASK)

In [ ]:
EMBEDDING = RBTL_DATA[["xi1", "xi2", "xi3"]].values
RBTL_RESIDUALS = RBTL_DATA["delta_m"].values

In [ ]:
def isomap(spectra, num_neighbors, num_components):
    mean_spectrum = np.mean(spectra, axis=0)
    fractional_differences = spectra / mean_spectrum - 1
    model = Isomap(n_neighbors=num_neighbors, n_components=num_components)
    embedding = model.fit_transform(fractional_differences)
    embedding[:, 1] *= -1
    return embedding

ISOMAP_EMBEDDING = isomap(SPECTRA, 10, 3)

In [ ]:
def t_sne(spectra, num_components, perplexity, mask=None):
    mean_spectrum = np.mean(spectra, axis=0)
    fractional_differences = spectra / mean_spectrum - 1
    model = TSNE(n_components=num_components, method='exact', random_state=42, verbose=1, n_jobs=6, init="pca", n_iter=10000, perplexity=perplexity)
    if mask is not None:
        fractional_differences = fractional_differences[mask]
    embedding = model.fit_transform(fractional_differences)
    return embedding

TSNE_EMBEDDING = t_sne(SPECTRA, 3, 50, mask=NOISE_MASK)

In [ ]:
scatter_combined(
    EMBEDDING, RBTL_RESIDUALS,
    "./figures/rbtl-disp.png", vmin=-0.5, vmax=0.5, invert_colorbar=True, mask=NOISE_MASK
)

In [ ]:
scatter_combined(
    ISOMAP_EMBEDDING, RBTL_RESIDUALS,
    "./figures/rbtl-disp-mine.png", vmin=-0.5, vmax=0.5, invert_colorbar=True, mask=NOISE_MASK
)

In [ ]:
scatter_combined(
    TSNE_EMBEDDING, RBTL_RESIDUALS[NOISE_MASK],
    "./figures/rbtl-disp-tsne.png", vmin=-0.5, vmax=0.5, invert_colorbar=True
)

In [144]:
def nmad(x, *args, unbiased=False, centered=False, **kwargs):
    x = np.asarray(x)
    if not centered:
        x = x - np.median(x, *args, **kwargs)

    res = 1.4826 * np.median(np.abs(x), *args, **kwargs)

    if unbiased:
        res = res * x.size / (x.size - 1)

    return res

In [145]:
print(np.std(RBTL_RESIDUALS[COMBINED_MASK]))
print(nmad(RBTL_RESIDUALS[COMBINED_MASK]))
print(COMBINED_MASK.sum())

0.193943567386762
0.11564279999999999
156


In [ ]:
RBTL_RESIDUALS[np.logical_and(RBTL_RESIDUALS < 0.5, RBTL_RESIDUALS > -0.5)].shape

In [ ]:
sns.histplot(RBTL_RESIDUALS[NOISE_MASK])